In [2]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1, 2"

import torch
print(torch.backends.cuda.is_built())
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


/home/chaeeun/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
True
cuda


In [ ]:
## Models to run
# bioGPT
# bioGPT large
# Flan-T5
# google/pegasus-pubmed

In [ ]:
from transformers import AutoConfig, AutoModelForSeq2SeqLM, AutoTokenizer
import json

# model_name = 'google/flan-t5-small'
# model_name = 'google/flan-t5-base'
# model_name = 'google/flan-t5-small'
# model_name = 'google/flan-t5-xl'
model_name = 'google/flan-t5-large'
# model_name = 'google/flan-t5-xxl'
# model_name = "google/pegasus-pubmed"

config = AutoConfig.from_pretrained(model_name)
config.n_positions = 2048
tokenizer = AutoTokenizer.from_pretrained(model_name)

if model_name == "microsoft/BioGPT-Large":
    model = AutoModelForCausalLM.from_pretrained("microsoft/BioGPT-Large")
else:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=config)
    
model.to(device)

In [14]:
## 교수님께서 주신 예시

item1 = """Here we report the characterization of the human Notch3 gene which we mapped to the CADASIL critical region. We have identified mutations in CADASIL patients that cause serious disruption of this gene, indicating that Notch3 could be the defective protein in CADASIL patients.All these missense mutations may result in severe disruption of the Notch3 protein, as suggested by the highly conserved nature of the aminoacid residues involved, particularly the cysteines that are key features of EGF likedomains24. These results indicate that these nucleotide substitutions are pathogenic mutations rather than rare polymorphisms.[1]"""
item2 = """Linkage studies in other families enabled further refinement of this genetic interval16,17 and identification of the mutated gene as NOTCH3 (Notch homolog 3).18[2]"""
item3 = """CADASIL, a hereditary vascular dementia, suggesting a role for Notch3 in vessel homeostasis (Joutel et al. 1996). CADASIL is a late-onset disorder, and neurological symptoms arise from a slowly developing systemic vasculopathy, characterized ultimately by degeneration of VSMC. ...maturation of VSMC, and ends around P28 when the artery acquires its final shape. We identify Notch3 to be the first key player of this process, by regulating cell-autonomously the arterial differentiation and maturation of VSMC.[3]"""

item4 = "There is currently no consensus on whether CADASIL mutations generate hyperactive or hypoactive Notch3 proteins with regard to downstream signaling or whether CADASIL mutations are neutral in terms of Notch signaling. Whereas the R169C mutation appeared to lead to hyperactive Notch signaling (see above), the R1031C or C455R mutations were instead shown to be hypoactive,36[1]"
item5 = "We believe that increased TGFb3 reflects an inflammatory condition and even an involvement of TGFb in fibrosis in CADASIL. Support for our hypothesis is the outcome of several studies that showed NOTCH3 and TGFb1 signalling play a key role in the pathogenesis and progression of chronic cardiovascular disease.[2]"
item6 = "Over time, VSMCs apoptosis leads to fibrosis and thickening of the arterial wall, progressive lumen stenosis and vascular insufficiency that makes the already poorly perfused terminal regions particularly susceptible to infarcts. SMCs degeneration is followed by the emergence of large,...[3]"
item7 = "Previously published studies have shown that TGF-b signaling is closely associated with the activity of SYK, and the kinase activity of SYK is essential for the activation of some signaling receptor downstream effector molecules[25]. ...bidirectional[33]. In the present study, the activation of SYK was shown to increase the progression of peritoneal fibrosis through activation of the TGF-b1/Smad3 signaling pathway, and inhibition of TGF-b1 also resulted in down-regulation of SYK.[4]"
item8 = "Our work confirms and highlights the relevance of NOTCH3 expression and signaling in pro-inflammatory macrophage activation and identified its prominent and specific role in the activation of NF-kB. A positive regulation between NOTCH and NF-kB signaling pathway has been described previously in macrophages isolated from patients with atherosclerosis. In those patients, and in contrast with our results,[5]"
item9 = "Niclosamide and derivatives - potent inhibitors of TMEM16A/F and Ca2+ signaling. NFA is a nonsteroidal anti-inflammatory compound that inhibits goblet cell degranulation and suppresses asthma phenotype (4, 27). ...This result suggests an antiinflammatory effect of niclosamide, which corresponds well to inhibition of allergic lung inflammation by the TMEM16A inhibitor benzbromarone (3, 31).[6]"
item10 = "The key findings of this study are that (a) Cl- fluxes mediated by the Ca2+gated channel TMEM16A are a crucial determinant of pericyte tone; (b) TMEM16A is activated during ischemia and evokes a long-lasting pericytemediated capillary constriction that reduces CBF and favors neutrophil and platelet stalling; (c) genetic analysis suggests that increased TMEM16A expression is associated with poor recovery after ischemic stroke (and the genetic proxy[7]"

item11 = "Role of NOTCH3 Mutations in the Cerebral Small Vessel Disease Cerebral Autosomal Dominant Arteriopathy ... (Stroke 2018). There is currently no consensus on whether CADASIL mutations generate hyperactive or hypoactive Notch3 proteins with ...[1]"
item12 = "Differences in proliferation rate between CADASIL and control vascular smooth muscle cells are ... (J Cell Mol Med 2018). We believe that increased TGFb3 reflects an inflammatory condition and even an involvement of TGFb in fibrosis in CADASIL. ... showed NOTCH3 and TGFb1 signalling play a key role in the pathogenesis and progression of chronic...[2]"
item13 = "The Role of Vascular Smooth Muscle Cells in Arterial Remodeling: Focus on Calcification ... (Int J Mol Sci 2019). Over time, VSMCs apoptosis leads to fibrosis and thickening of the arterial wall, progressive lumen stenosis and vascular insufficiency that makes the already poorly perfused ...[3]"
item14 = "Spleen Tyrosine Kinase (SYK) in the Progression of Peritoneal Fibrosis Through Activation of the ... (Med Sci Monit 2019). Previously published studies have shown that TGF-b signaling is closely associated with the activity of SYK, and the kinase ...[4]"
item15 = "NOTCH3 signaling is essential for NF-kB activation in TLRactivated macrophages (Sci Rep 2020). Our work confirms and highlights the relevance of NOTCH3 expression and signaling in pro-inflammatory macrophage ...[5]"
item16 = "Niclosamide repurposed for the treatment of inflammatory airway disease (JCI Insight 2019). Niclosamide and derivatives - potent inhibitors of TMEM16A/F and Ca2+ signaling. NFA is a nonsteroidal antiinflammatory compound that inhibits goblet ...[6]"
item17 = "The Ca2+-gated channel TMEM16A amplifies capillary pericyte contraction and reduces ... (J Clin Invest. 2022). The key findings of this study are that (a) Cl- fluxes mediated by the Ca2+-gated channel TMEM16A are a crucial determinant of pericyte tone[7]"

gt1 = "CADASIL is a rare hereditary cerebrovascular disease caused by a NOTCH3 mutation[1][2]. NOTCH3 is particularly important for the maturation of vascular smooth muscle cells (VSMCs). A decreased VSMC is a pathological hallmark for CADASIL[3]."
gt2 = "Syk is a therapeutic target for CADASIL. In CADASIL, proinflammatory signaling is activated[1] through the increase of TGFb1 signaling[2]. The apoptosis of vascular smooth muscle cell (VSMC) and increased TGFb1 signaling cause fibrosis and vascular abnormalities in vascular epithelial cells[3]. Syk increases fibrosis through activation of TGFb1 signaling[4]."
gt3 = "TMEM16A is a therapeutic target for CADASIL. Ischemia in CADASIL activates TMEM16A, Ca2+ gated Cl- channel, which regulates pericyte tone and induces capillary constriction[1]. Potent inhibitors of TMEM16A, niclosamide has antiinflammtory effect[2]. Niclosamide can suppress enhance inflammation in CADASIL[3]."
gt4 = "Syk is a therapeutic target for CADASIL. In CADASIL, proinflammatory signaling is activated[1] through the increase of TGFb1 signaling[2] . The apoptosis of vascular smooth muscle cell (VSMC) and increased TGFb1 signaling cause fibrosis and vascular abnormalities in vascular epithelial cells[3] . Syk increases fibrosis through activation of TGFb1 signaling[4]."
gt5 = "TMEM16A is a therapeutic target for CADASIL. Ischemia in CADASIL activates TMEM16A, Ca2+ gated Cl- channel, which regulates pericyte tone and induces capillary constriction[1]. Potent inhibitors of TMEM16A, niclosamide has antiinflammtory effect[2]. Niclosamide can suppress enhance inflammation in CADASIL[3]."

q1 = "What is the cause of the CADASIL?"
q2 = "What is the therapeutic target for CADASIL?"
q3 = "What is the therapeutic target for CADASIL?"

In [15]:
## 현재님께서 retrieve하신 evidences

rt_item1 = "CADASIL) is sometimes misdiagnosed as multiple sclerosis (MS). MS and CADASIL are not known to co-occur and brain magnetic resonance imagining (MRI) findings can help with differential diagnosis. Despite the availability of this information, a case report is presented of a 61-year-old woman who was misdiagnosed with MS at age 50, tested positive for CADASIL at age 56, described incorrectly as having both conditions simultaneously, and continued on MS disease-modifying medications, resulting in financial and physical hardship.[1]"
rt_item2 = "Despite the availability of this information, a case report is presented of a 61-year-old woman who was misdiagnosed with MS at age 50, tested positive for CADASIL at age 56, described incorrectly as having both conditions simultaneously, and continued on MS disease-modifying medications, resulting in financial and physical hardship. Neuropsychological consultation helped initiate removal of the MS diagnosis and treatment. Better understanding is needed among clinicians that MS and CADASIL are not known to co-exist, that no association has been found between MS and the NOTCH3 mutations that cause CADASIL, and that neuroimaging and clinical features can help distinguish between the two conditions in addition to genetic testing.[2]"
rt_item3 = "Antithrombotic drugs were used in three patients, in one for an unrelated coexisting prothrombotic condition. CADASIL does not seem to be associated with an unfavorable outcome of pregnancy either for women and fetuses. Patients and treating physicians should be reassured that pregnancy can be safely initiated in CADASIL, as there is no evidence to support a specific preventive antithrombotic treatment during pregnancy in CADASIL.[3]"

rt_item4 = "Using both wild type mice and transgenic mice carrying the human mutant Notch3 gene (CADASIL mice), we have recently characterized the pathological features of CADASIL and determined the therapeutic efficacy of two hematopoietic growth factors, stem cell factor (SCF) and granulocyte colony-stimulating factor (G-CSF) in CADASIL. Our findings have revealed novel pathological changes in the endothelium of cerebral capillaries and in the neural stem cells (NSCs). We have also observed the impairment of cognitive function in CADASIL mice.[1]"
rt_item5 = "Cerebral autosomal dominant arteriopathy with subcortical infarcts and leukoencephalopathy (CADASIL) is the most common condition of hereditary stroke and vascular dementia. CADASIL is caused by Notch3 mutation, leading to progressive degeneration of vascular smooth muscle cells (VSMCs) of the small arteries in the brain. However, the pathogenesis of CADASIL remains largely unknown, and treatment that can stop or delay the progression of CADASIL is not yet available.[2]"
rt_item6 = "Today, CADASIL is known to every neurologist, but the disease has not yet revealed all its secrets. A lot of effort is still needed to understand the intimate mechanisms of the disease and the most efficient targets or approaches for the development of efficient therapeutics. The history of CADASIL will be further enriched by multiple ongoing research projects worldwide, at clinical and preclinical level, and will continue to enlighten research in the field of cerebral small vessel disorders.[3]"

rt_item7 = "Syk inhibition was found to stabilize microtubules and potentiate paclitaxel activity in cellular models of taxane-resistant ovarian cancers. We further studied the effects of Syk inhibition on paclitaxel activity in Syk(+) ovarian cancer cell models and in variants selected for taxane resistance. Syk inhibition was accomplished using RNAi and by exposure to the small molecule competitive inhibitor R406, the active metabolite of fostamatinib.[1]"
rt_item8 = "Therefore, inhibition of the inflammatory macrophages would be a promising approach to attenuate NASH. In this study, we studied the implication of SYK pathway in NASH, and investigated PLGA nanoparticles-based delivery of SYK pathway inhibitor as an effective and promising therapeutic approach for the treatment of NASH. We found positive correlation between SYK expression with the pathogenesis of NASH and alcoholic hepatitis in patients.[2]"
rt_item9 = "Cerebral autosomal dominant arteriopathy with subcortical infarcts and leukoencephalopathy (CADASIL) is the most common condition of hereditary stroke and vascular dementia. CADASIL is caused by Notch3 mutation, leading to progressive degeneration of vascular smooth muscle cells (VSMCs) of the small arteries in the brain. However, the pathogenesis of CADASIL remains largely unknown, and treatment that can stop or delay the progression of CADASIL is not yet available.[3]"


rt_q1 = "can nilvadipine help alleviate cadasil symptoms?"
rt_q2 = "what are the potential therapeutic targets for cadasil?"
rt_q3 = "would SYK inhibition be potential treatment for cadasil?"

In [16]:
## dic['data'] = list of dictionaries. each dic has fields: "query", "evidences"(itself a list of strings), "ground_truth"
## pre-compute data dictionary file. yea? yep. 
## see how the code handles cases without ground truth -> from same example dic, just doesn't take the ground truth. 
## make three with complete examples, three with retrieved evidences, 
## or make two json files one with the three complete examples and the other with retrieved evidences. 

import json

context1 = {"query": q1, "evidences": [item1, item2, item3], "ground_truth": gt1}

context2 = {"query": q2, "evidences": [item4, item5, item6, item7, item8], "ground_truth": gt2}
context3 = {"query": q2, "evidences": [item4, item8, item9, item10], "ground_truth": gt3}

context4 = {"query": q3, "evidences": [item11, item12, item13, item14, item15], "ground_truth": gt4}
context5 = {"query": q3, "evidences": [item11, item15, item16, item17], "ground_truth": gt5}

query1 = {"query": rt_q1, "evidences": [rt_item1, rt_item2, rt_item3]}
query2 = {"query": rt_q2, "evidences": [rt_item4, rt_item5, rt_item6]}
query3 = {"query": rt_q3, "evidences": [rt_item7, rt_item8, rt_item9]}

dic = {'contexts': [context1, context2, context3, context4, context5], 'queries': [query1, query2, query3]}

with open("examples_20230207.json", "w") as outfile:
    json.dump(dic, outfile)

In [17]:
with open('examples_20230207.json') as f:
    loaded_dic = json.load(f)
    contexts = loaded_dic['contexts']
    queries = loaded_dic['queries']
    
    
## any combination of contexts[i] and queries[i] possible? -> well contexts can be query but not the other way around
# also, context 23 and 45 are v similar

args = [(contexts[0], contexts[1]), (contexts[0], contexts[2]), (contexts[0], contexts[3]), (contexts[0], contexts[4]),\
        (contexts[1], contexts[0]), (contexts[2], contexts[0]), (contexts[3], contexts[0]), (contexts[4], contexts[0]),\
        (contexts[1], contexts[2]), (contexts[2], contexts[1]),\
        (contexts[3], contexts[4]), (contexts[4], contexts[3]),\
        (contexts[0], queries[0]), (contexts[1], queries[0]), (contexts[2], queries[0]),\
        (contexts[0], queries[1]), (contexts[1], queries[1]), (contexts[2], queries[1]),\
        (contexts[0], queries[2]), (contexts[1], queries[2]), (contexts[2], queries[2])]

args = []
for i, c1 in enumerate(contexts):
    for j, c2 in enumerate(contexts):
        if i != j:
            args.append((c1, c2))
    for q in queries:
        args.append((c1, q))


In [9]:
def make_prompt(example, query, zeroshot=False): # both args are of type either context dic or query dic -> {"query": q3, "evidences": [item11, item15, item16, item17], "ground_truth": gt5}
    
    example_part = ""
    if not zeroshot:
        for i, ev in enumerate(example['evidences']):
            example_part += f"evidence {i}: {ev} \n"

        example_part += f"query: {example['query']} \n"
        example_part += "summarize given evidences and query:"
        example_part += f"{example['ground_truth']} \n\n"
    
    query_part = ""
    for i, ev in enumerate(query['evidences']):
        query_part += f"evidence {i}: {ev} \n"
    
    query_part += f"query: {query['query']} \n"
    query_part += "summarize given evidences and query:"
    
    prompt =  example_part + query_part
    return prompt



In [10]:
tokenizer.to(device)

AttributeError: 'PegasusTokenizerFast' object has no attribute 'to'

In [11]:
item1 = '''Here we report the characterization of the human Notch3 gene which we mapped to the CADASIL critical region. We have identified mutations in CADASIL patients that cause serious disruption of this gene, indicating that Notch3 could be the defective protein in CADASIL patients. All these missense mutations may result in severe disruption of the Notch3 protein, as suggested by the highly conserved nature of the aminoacid residues involved, particularly the cysteines that are key features of EGF likedomains24. These results indicate that these nucleotide substitutions are pathogenic mutations rather than rare polymorphisms.'''
item2 = '''Linkage studies in other families enabled further refinement of this genetic interval16,17 and identification of the mutated gene as NOTCH3 (Notch homolog 3).18'''
item3 = '''CADASIL, a hereditary vascular dementia, suggesting a role for Notch3 in vessel homeostasis (Joutel et al. 1996). CADASIL is a late-onset disorder, and neurological symptoms arise from a slowly developing systemic vasculopathy, characterized ultimately by degeneration of vSMC . maturation of vSMC, and ends around P28 when the artery acquires its final shape. We identify Notch3 to be the first key player of this process, by regulating cell-autonomously the arterial differentiation and maturation of vSMC.'''

item3 = "There is currently no consensus on whether CADASIL mutations generate hyperactive or hypoactive Notch3 proteins with regard to downstream signaling or whether CADASIL mutations are neutral in terms of Notch signaling. Whereas the R169C mutation appeared to lead to hyperactive Notch signaling (see above), the R1031C or C455R mutations were instead shown to be hypoactive,36 "
item4 = "We believe that increased TGFb3 reflects an inflammatory condition and even an involvement of TGFb in fibrosis in CADASIL. Support for our hypothesis is the outcome of several studies that showed NOTCH3 and TGFb1 signalling play a key role in the pathogenesis and progression of chronic cardiovascular disease. "
item5 = "Over time, VSMCs apoptosis leads to fibrosis and thickening of the arterial wall, progressive lumen stenosis and vascular insufficiency that makes the already poorly perfused terminal regions particularly susceptible to infarcts. SMCs degeneration is followed by the emergence of large"
item6 = "Previously published studies have shown that TGF-b signaling is closely associated with the activity of SYK, and the kinase activity of SYK is essential for the activation of some signaling receptor downstream effector molecules. In the present study, the activation of SYK was shown to increase the progression of peritoneal fibrosis through activation of the TGF-b1/Smad3 signaling pathway, and inhibition of TGF-b1 also resulted in down-regulation of SYK."
item7 = "Our work confirms and highlights the relevance of NOTCH3 expression and signaling in pro-inflammatory macrophage activation and identified its prominent and specific role in the activation of NF-κB. A positive regulation between NOTCH and NF-κB signaling pathway has been described previously in macrophages isolated from patients with atherosclerosis. In those patients, and in contrast with our results, "

gt = "Syk is a therapeutic target for CADASIL. In CADASIL, proinflammatory signaling is activated through the increase of TGFb1 signaling. The apoptosis of vascular smooth muscle cell (VSMC) and increased TGFb1 signaling cause fibrosis and vascular abnormalities in vascular epithelial cells. Syk increases fibrosis through activation of TGF-b1 signaling."

set1 = {"query": 'What is the cause of the CADASIL?', "evidences": [item1, item2, item3]}
set2 = {"query": 'What is the therapeutic target for CADASIL?', "evidences": [item3, item4, item5, item6, item7], "ground_truth": gt}

args = (set2, set1)


In [19]:

with open(f"generation_results_{model_name.replace('/', '-')}_one-shot.txt", "w") as results: # mujeen_
    results.write(f'<< Model: {model_name} >>\n\n')
    # results.write()

    for i in range(len(args)):
    # for i in range(1):
        example, query = args[i] # 무진님꺼할땐 indexing 빼기
        prompt = make_prompt(example, query)
        
        print('< PROMPT >\n')
        print(prompt)
        results.write('< PROMPT >\n\n')
        results.write(prompt)
        
        # inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        inputs.to(device)
        outputs = model.generate(**inputs, max_length=1024)
        generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        print('\n\n< GENERATED >\n')
        print(generated)
        print('\n###############################################################################################################################################################\n')
        results.write('\n\n< GENERATED >\n\n')
        results.write(*generated)
        results.write('\n\n###############################################################################################################################################################\n')
        
results.close()
    

< PROMPT >

evidence 0: Here we report the characterization of the human Notch3 gene which we mapped to the CADASIL critical region. We have identified mutations in CADASIL patients that cause serious disruption of this gene, indicating that Notch3 could be the defective protein in CADASIL patients.All these missense mutations may result in severe disruption of the Notch3 protein, as suggested by the highly conserved nature of the aminoacid residues involved, particularly the cysteines that are key features of EGF likedomains24. These results indicate that these nucleotide substitutions are pathogenic mutations rather than rare polymorphisms.[1] 
evidence 1: Linkage studies in other families enabled further refinement of this genetic interval16,17 and identification of the mutated gene as NOTCH3 (Notch homolog 3).18[2] 
evidence 2: CADASIL, a hereditary vascular dementia, suggesting a role for Notch3 in vessel homeostasis (Joutel et al. 1996). CADASIL is a late-onset disorder, and neur

In [13]:
with open(f"generation_results_mujeen_{model_name.replace('/', '-')}_zero-shot.txt", "w") as results:
    results.write(f'<< Model: {model_name} >>\n\n')
    # results.write()

    # for i in range(len(args)):
    for i in range(1):
        example, query = args #[i]
        prompt = make_prompt(example, query, zeroshot=True)
        
        print('< PROMPT >\n')
        print(prompt)
        results.write('< PROMPT >\n\n')
        results.write(prompt)
        
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        inputs.to(device)
        outputs = model.generate(**inputs, max_length=1024)
        generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        print('\n\n< GENERATED >\n')
        print(generated)
        print('\n###############################################################################################################################################################\n')
        results.write('\n\n< GENERATED >\n\n')
        results.write(*generated)
        results.write('\n\n###############################################################################################################################################################\n')
        
results.close()

< PROMPT >

evidence 0: Here we report the characterization of the human Notch3 gene which we mapped to the CADASIL critical region. We have identified mutations in CADASIL patients that cause serious disruption of this gene, indicating that Notch3 could be the defective protein in CADASIL patients. All these missense mutations may result in severe disruption of the Notch3 protein, as suggested by the highly conserved nature of the aminoacid residues involved, particularly the cysteines that are key features of EGF likedomains24. These results indicate that these nucleotide substitutions are pathogenic mutations rather than rare polymorphisms. 
evidence 1: Linkage studies in other families enabled further refinement of this genetic interval16,17 and identification of the mutated gene as NOTCH3 (Notch homolog 3).18 
evidence 2: There is currently no consensus on whether CADASIL mutations generate hyperactive or hypoactive Notch3 proteins with regard to downstream signaling or whether CA

In [9]:

# inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
# outputs = model.generate(**inputs, max_length=2048)
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [10]:
mujeen_input_text = """evidence 0: There is currently no consensus on whether CADASIL mutations generate hyperactive or hypoactive Notch3 proteins with regard to downstream signaling or whether CADASIL mutations are neutral in terms of Notch signaling. Whereas the R169C mutation appeared to lead to hyperactive Notch signaling (see above), the R1031C or C455R mutations were instead shown to be hypoactive,36 
evidence 1: We believe that increased TGFb3 reflects an inflammatory condition and even an involvement of TGFb in fibrosis in CADASIL. Support for our hypothesis is the outcome of several studies that showed NOTCH3 and TGFb1 signalling play a key role in the pathogenesis and progression of chronic cardiovascular disease. 
evidence 2: Over time, VSMCs apoptosis leads to fibrosis and thickening of the arterial wall, progressive lumen stenosis and vascular insufficiency that makes the already poorly perfused terminal regions particularly susceptible to infarcts. SMCs degeneration is followed by the emergence of large 
evidence 3: Previously published studies have shown that TGF-b signaling is closely associated with the activity of SYK, and the kinase activity of SYK is essential for the activation of some signaling receptor downstream effector molecules. In the present study, the activation of SYK was shown to increase the progression of peritoneal fibrosis through activation of the TGF-b1/Smad3 signaling pathway, and inhibition of TGF-b1 also resulted in down-regulation of SYK. 
evidence 4: Our work confirms and highlights the relevance of NOTCH3 expression and signaling in pro-inflammatory macrophage activation and identified its prominent and specific role in the activation of NF-κB. A positive regulation between NOTCH and NF-κB signaling pathway has been described previously in macrophages isolated from patients with atherosclerosis. In those patients, and in contrast with our results, 
query: What is the therapeutic target for CADASIL? 
summarize given evidences and query: Syk is a therapeutic target for CADASIL. In CADASIL, proinflammatory signaling is activated through the increase of TGFb1 signaling. The apoptosis of vascular smooth muscle cell (VSMC) and increased TGFb1 signaling cause fibrosis and vascular abnormalities in vascular epithelial cells. Syk increases fibrosis through activation of TGF-b1 signaling. 

evidence 0: Here we report the characterization of the human Notch3 gene which we mapped to the CADASIL critical region. We have identified mutations in CADASIL patients that cause serious disruption of this gene, indicating that Notch3 could be the defective protein in CADASIL patients. All these missense mutations may result in severe disruption of the Notch3 protein, as suggested by the highly conserved nature of the aminoacid residues involved, particularly the cysteines that are key features of EGF likedomains24. These results indicate that these nucleotide substitutions are pathogenic mutations rather than rare polymorphisms. 
evidence 1: Linkage studies in other families enabled further refinement of this genetic interval16,17 and identification of the mutated gene as NOTCH3 (Notch homolog 3).18 
evidence 2: CADASIL, a hereditary vascular dementia, suggesting a role for Notch3 in vessel homeostasis (Joutel et al. 1996). CADASIL is a late-onset disorder, and neurological symptoms arise from a slowly developing systemic vasculopathy, characterized ultimately by degeneration of vSMC . maturation of vSMC, and ends around P28 when the artery acquires its final shape. We identify Notch3 to be the first key player of this process, by regulating cell-autonomously the arterial differentiation and maturation of vSMC. 
query: What is the cause of the CADASIL? 
summarize given evidences and query:"""

In [31]:
evidence_input = """Evidence: Here we report the characterization of the human Notch3 gene which we mapped to the CADASIL critical region. We have identified mutations in CADASIL patients that cause serious disruption of this gene, indicating that Notch3 could be the defective protein in CADASIL patients.All these missense mutations may result in severe disruption of the Notch3 protein, as suggested by the highly conserved nature of the aminoacid residues involved, particularly the cysteines that are key features of EGF likedomains24. These results indicate that these nucleotide substitutions are pathogenic mutations rather than rare polymorphisms.
Summarize the evidence: In the evidence, it is mentioned that mutations were identified in CADASIL patients and that it may result in severe disruption of the Notch3 protein.

Evidence: CADASIL, a hereditary vascular dementia, suggesting a role for Notch3 in vessel homeostasis (Joutel et al. 1996). CADASIL is a late-onset disorder, and neurological symptoms arise from a slowly developing systemic vasculopathy, characterized ultimately by degeneration of vSMC . maturation of vSMC, and ends around P28 when the artery acquires its final shape. We identify Notch3 to be the first key player of this process, by regulating cell-autonomously the arterial differentiation and maturation of vSMC. 
Summarize the evidence:
"""

In [14]:
input_text = """evidence 0: There is currently no consensus on whether CADASIL mutations generate hyperactive or hypoactive Notch3 proteins with regard to downstream signaling or whether CADASIL mutations are neutral in terms of Notch signaling. Whereas the R169C mutation appeared to lead to hyperactive Notch signaling (see above), the R1031C or C455R mutations were instead shown to be hypoactive,36 
evidence 1: We believe that increased TGFb3 reflects an inflammatory condition and even an involvement of TGFb in fibrosis in CADASIL. Support for our hypothesis is the outcome of several studies that showed NOTCH3 and TGFb1 signalling play a key role in the pathogenesis and progression of chronic cardiovascular disease. 
evidence 2: Over time, VSMCs apoptosis leads to fibrosis and thickening of the arterial wall, progressive lumen stenosis and vascular insufficiency that makes the already poorly perfused terminal regions particularly susceptible to infarcts. SMCs degeneration is followed by the emergence of large 
evidence 3: Previously published studies have shown that TGF-b signaling is closely associated with the activity of SYK, and the kinase activity of SYK is essential for the activation of some signaling receptor downstream effector molecules. In the present study, the activation of SYK was shown to increase the progression of peritoneal fibrosis through activation of the TGF-b1/Smad3 signaling pathway, and inhibition of TGF-b1 also resulted in down-regulation of SYK. 
evidence 4: Our work confirms and highlights the relevance of NOTCH3 expression and signaling in pro-inflammatory macrophage activation and identified its prominent and specific role in the activation of NF-κB. A positive regulation between NOTCH and NF-κB signaling pathway has been described previously in macrophages isolated from patients with atherosclerosis. In those patients, and in contrast with our results, 
query: What is the therapeutic target for CADASIL? 
summarize given evidences and query: Syk is a therapeutic target for CADASIL. In CADASIL, proinflammatory signaling is activated through the increase of TGFb1 signaling. The apoptosis of vascular smooth muscle cell (VSMC) and increased TGFb1 signaling cause fibrosis and vascular abnormalities in vascular epithelial cells. Syk increases fibrosis through activation of TGF-b1 signaling. 

evidence 0: CADASIL) is sometimes misdiagnosed as multiple sclerosis (MS). MS and CADASIL are not known to co-occur and brain magnetic resonance imagining (MRI) findings can help with differential diagnosis. Despite the availability of this information, a case report is presented of a 61-year-old woman who was misdiagnosed with MS at age 50, tested positive for CADASIL at age 56, described incorrectly as having both conditions simultaneously, and continued on MS disease-modifying medications, resulting in financial and physical hardship.
evidence 1: Despite the availability of this information, a case report is presented of a 61-year-old woman who was misdiagnosed with MS at age 50, tested positive for CADASIL at age 56, described incorrectly as having both conditions simultaneously, and continued on MS disease-modifying medications, resulting in financial and physical hardship. Neuropsychological consultation helped initiate removal of the MS diagnosis and treatment. Better understanding is needed among clinicians that MS and CADASIL are not known to co-exist, that no association has been found between MS and the NOTCH3 mutations that cause CADASIL, and that neuroimaging and clinical features can help distinguish between the two conditions in addition to genetic testing.
evidence 2: Antithrombotic drugs were used in three patients, in one for an unrelated coexisting prothrombotic condition. CADASIL does not seem to be associated with an unfavorable outcome of pregnancy either for women and fetuses. Patients and treating physicians should be reassured that pregnancy can be safely initiated in CADASIL, as there is no evidence to support a specific preventive antithrombotic treatment during pregnancy in CADASIL.
query: can nilvadipine help alleviate cadasil symptoms?
summarize given evidences and query:"""

In [27]:
input_text = """evidence 0: Here we report the characterization of the human Notch3 gene which we mapped to the CADASIL critical region. We have identified mutations in CADASIL patients that cause serious disruption of this gene, indicating that Notch3 could be the defective protein in CADASIL patients.All these missense mutations may result in severe disruption of the Notch3 protein, as suggested by the highly conserved nature of the aminoacid residues involved, particularly the cysteines that are key features of EGF likedomains24. These results indicate that these nucleotide substitutions are pathogenic mutations rather than rare polymorphisms. 
evidence 1: Linkage studies in other families enabled further refinement of this genetic interval16,17 and identification of the mutated gene as NOTCH3 (Notch homolog 3).18 
evidence 2: CADASIL, a hereditary vascular dementia, suggesting a role for Notch3 in vessel homeostasis (Joutel et al. 1996). CADASIL is a late-onset disorder, and neurological symptoms arise from a slowly developing systemic vasculopathy, characterized ultimately by degeneration of VSMC . maturation of vSMC, and ends around P28 when the artery acquires its final shape. We identify Notch3 to be the first key player of this process, by regulating cell-autonomously the arterial differentiation and maturation of vSMC.
query: What is the cause of the CADASIL?
summarize given evidences given the query: In evidence 0, it is mentioned that mutations were identified in CADASIL patients and that it may result in severe disruption of the Notch3 protein. In evidence 1, it is mentioned that the mutated gene was identified as NOTCH3. In evidence 2, it is mentioned that Notch3 is the first key player in the maturation of VSMCs.

evidence 0: “Role of NOTCH3 Mutations in the Cerebral Small Vessel Disease Cerebral Autosomal Dominant Arteriopathy ... (Stroke 2018)” There is currently no consensus on whether CADASIL mutations generate hyperactive or hypoactive Notch3 proteins with …
evidence 1: “Differences in proliferation rate between CADASIL and control vascular smooth muscle cells are ... (J Cell Mol Med 2018)” We believe that increased TGFb3 reflects an inflammatory condition and even an involvement of TGFb in fibrosis in CADASIL. … showed NOTCH3 and TGFb1 signalling play a key role in the pathogenesis and progression of chronic …
evidence 2: “The Role of Vascular Smooth Muscle Cells in Arterial Remodeling: Focus on Calcification ... (Int J Mol Sci 2019)” Over time, VSMCs apoptosis leads to fibrosis and thickening of the arterial wall, progressive lumen stenosis and vascular insufficiency that makes the already poorly perfused …
query: What is the therapeutic target for CADASIL?
summarize given evidences given the query: """

In [17]:
input_text = """Evidence 0: Here we report the characterization of the human Notch3 gene which we mapped to the CADASIL critical region. We have identified mutations in CADASIL patients that cause serious disruption of this gene, indicating that Notch3 could be the defective protein in CADASIL patients.All these missense mutations may result in severe disruption of the Notch3 protein, as suggested by the highly conserved nature of the aminoacid residues involved, particularly the cysteines that are key features of EGF likedomains24. These results indicate that these nucleotide substitutions are pathogenic mutations rather than rare polymorphisms. 
Evidence 1: Linkage studies in other families enabled further refinement of this genetic interval16,17 and identification of the mutated gene as NOTCH3 (Notch homolog 3).18 
Evidence 2: CADASIL, a hereditary vascular dementia, suggesting a role for Notch3 in vessel homeostasis (Joutel et al. 1996). CADASIL is a late-onset disorder, and neurological symptoms arise from a slowly developing systemic vasculopathy, characterized ultimately by degeneration of VSMC . maturation of vSMC, and ends around P28 when the artery acquires its final shape. We identify Notch3 to be the first key player of this process, by regulating cell-autonomously the arterial differentiation and maturation of vSMC.
Query: What is the cause of the CADASIL?
Summarization of evidences given the query:
CADASIL is a rare hereditary cerebrovascular disease caused by a NOTCH3 mutation[1][2]. NOTCH3 is particularly important for the maturation of vascular smooth muscle cells (VSMCs). A decreased VSMC is a pathological hallmark for CADASIL[3].

Evidence 0: “Role of NOTCH3 Mutations in the Cerebral Small Vessel Disease Cerebral Autosomal Dominant Arteriopathy … (Stroke 2018)” There is currently no consensus on whether CADASIL mutations generate hyperactive or hypoactive Notch3 proteins with …
Evidence 1: “Differences in proliferation rate between CADASIL and control vascular smooth muscle cells are … (J Cell Mol Med 2018)” We believe that increased TGFb3 reflects an inflammatory condition and even an involvement of TGFb in fibrosis in CADASIL. … showed NOTCH3 and TGFb1 signalling play a key role in the pathogenesis and progression of chronic …
Evidence 2: “The Role of Vascular Smooth Muscle Cells in Arterial Remodeling: Focus on Calcification … (Int J Mol Sci 2019)” Over time, VSMCs apoptosis leads to fibrosis and thickening of the arterial wall, progressive lumen stenosis and vascular insufficiency that makes the already poorly perfused …
Evidence 3: “Spleen Tyrosine Kinase (SYK) in the Progression of Peritoneal Fibrosis Through Activation of the … (Med Sci Monit 2019)” Previously published studies have shown that TGF-b signaling is closely associated with the activity of SYK, and the kinase …
Evidence 4: “NOTCH3 signaling is essential for NF-κB activation in TLRactivated macrophages (Sci Rep 2020)” Our work confirms and highlights the relevance of NOTCH3 expression and signaling in pro-inflammatory macrophage …
Evidence 5: “Niclosamide repurposed for the treatment of inflammatory airway disease (JCI Insight 2019)” Niclosamide and derivatives — potent inhibitors of TMEM16A/F and Ca2+ signaling. NFA is a nonsteroidal antiinflammatory compound that inhibits goblet …
Evidence 6: “The Ca2+-gated channel TMEM16A amplifies capillary pericyte contraction and reduces … (J Clin Invest. 2022)” The key findings of this study are that (a) Cl– fluxes mediated by the Ca2+-gated channel TMEM16A are a crucial determinant of pericyte tone
Query: What is the therapeutic target for CADASIL?
Summarization of evidences given the query: """